In [1]:
import toolbox as tb
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [30]:
movies_df = pd.read_csv("../wth_actors_name/data/all_movie_data.csv")

In [31]:
movies_df = movies_df.iloc[322:513]

In [32]:
movies_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers
322,tt0870111,movie,Frost/Nixon,Frost/Nixon,0,2008,\N,122,"Biography,Drama,History",7.7,103616.0,nm0000165,nm0604948
323,tt0824747,movie,Changeling,Changeling,0,2008,\N,141,"Biography,Crime,Drama",7.7,240132.0,nm0000142,nm0833089
324,tt0455590,movie,The Last King of Scotland,The Last King of Scotland,0,2006,\N,123,"Biography,Drama,History",7.7,175999.0,nm0531817,"nm0604948,nm0110591,nm1894768,nm9114061"
325,tt0421082,movie,Control,Control,0,2007,\N,122,"Biography,Drama,Music",7.7,61779.0,nm0179221,"nm1722281,nm0339043"
326,tt0119008,movie,Donnie Brasco,Donnie Brasco,0,1997,\N,127,"Biography,Crime,Drama",7.7,281237.0,nm0001565,"nm0685587,nm0940361,nm0001921"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,tt0101889,movie,The Fisher King,The Fisher King,0,1991,\N,137,"Comedy,Drama,Fantasy",7.5,79738.0,nm0000416,nm0481418
509,tt0097351,movie,Field of Dreams,Field of Dreams,0,1989,\N,107,"Drama,Family,Fantasy",7.5,108469.0,nm0004675,"nm0455875,nm0004675"
510,tt0094721,movie,Beetlejuice,Beetle Juice,0,1988,\N,92,"Comedy,Fantasy",7.5,264916.0,nm0000318,"nm0568313,nm0933733,nm0803730"
511,tt9484998,movie,Palm Springs,Palm Springs,0,2020,\N,90,"Comedy,Fantasy,Mystery",7.4,84287.0,nm3817317,nm4460791


In [34]:
movies_id_list = list(movies_df["tconst"])

ids_dict = {}
movie_counter = 321

for movie_id in movies_id_list:
            
    inner_dict = {}
    
    url = f"https://www.imdb.com/title/{movie_id}/"
    headers = {"Accept-Language": "en-US,en;q=0.5"}

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")


    # TOP 10 Actors Scraper
    cast_table = soup.find("table", class_="cast_list")
    mvp_actors = []
    counter = 0
    
    if cast_table == None: continue
        
    for row in cast_table.find_all("tr")[1:]:

        row_data = row.find_all("td")
        
        if len(row_data) < 4: break
            
        actor = (row_data[1].text)[2:-2]
        mvp_actors.append(actor)

        counter += 1

        if (counter == 10): break

    inner_dict["Actors"] = mvp_actors
    #print(mvp_actors)

    # Awards Scraper
    awards = soup.find_all("span", class_="awards-blurb")
    awards_blurb = []

    for blurb in awards:
        awards_blurb.append(blurb.text)

        
    inner_dict["Awards"] = awards_blurb
    #print(awards_blurb)

    # Budget Scraper

    for block in soup.find_all("div", class_="txt-block"):

        h4 = block.find("h4", class_="inline")

        if h4 != None:
            if h4.text == "Budget:":
                budget = block.text[8:].split("\n")[0]
                
                inner_dict["Budget"] = budget
                
    ids_dict[movie_id] = inner_dict
    
    movie_counter += 1
    print(movie_counter)
    
df = pd.DataFrame.from_dict(ids_dict, orient="index")

322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512


In [35]:
df

,Actors,Awards,Budget
tt0870111,"[Frank Langella, Michael Sheen, Sam Rockwell, ...",[\n\n Nominated for\n 5\...,"$25,000,000"
tt0824747,"[Angelina Jolie, Gattlin Griffith, Michelle Gu...",[\n\n Nominated for\n 3\...,"$55,000,000"
tt0455590,"[Forest Whitaker, James McAvoy, Kerry Washingt...",[\n\n Won\n 1\n ...,"$6,000,000"
tt0421082,"[Sam Riley, Samantha Morton, Alexandra Maria L...",[\n\n Nominated for\n 1\...,"EUR4,500,000"
tt0119008,"[Al Pacino, Johnny Depp, Michael Madsen, Bruno...",[\n\n Nominated for\n 1\...,"$35,000,000"
...,...,...,...
tt0101889,"[Jeff Bridges, Adam Bryant, Paul Lombardi, Dav...",[\n\n Won\n 1\n ...,"$24,000,000"
tt0097351,"[Kevin Costner, Amy Madigan, Gaby Hoffmann, Ra...",[\n\n Nominated for\n 3\...,NaN
tt0094721,"[Alec Baldwin, Geena Davis, Annie McEnroe, Mau...",[\n\n Won\n 1\n ...,"$15,000,000"
tt9484998,"[Andy Samberg, Cristin Milioti, J.K. Simmons, ...",[\n\n Nominated for\n 2\...,NaN


In [36]:
current_df = pd.read_csv("../wth_actors_name/data/scraped_movie_data.csv").set_index("Unnamed: 0")

In [37]:
current_df

,Actors,Awards,Budget
Unnamed: 0,,,
tt3040964,"['Neel Sethi', 'Bill Murray', 'Ben Kingsley', ...",['\n\n Won\n 1\n ...,"$175,000,000"
tt2310332,"['Ian McKellen', 'Martin Freeman', 'Richard Ar...",['\n\n Nominated for\n 1...,"$250,000,000"
tt1038988,"['Manuela Velasco', 'Ferran Terraza', 'Jorge-Y...",['\n 16 wins & 12 nominations.\n '],"EUR1,500,000"
tt0464141,"['Belén Rueda', 'Fernando Cayo', 'Roger Prínce...",['\n 32 wins & 43 nominations.\n '],"EUR3,400,000"
tt0451279,"['Gal Gadot', 'Chris Pine', 'Connie Nielsen', ...",['\n 25 wins & 71 nominations.\n '],"$149,000,000"
...,...,...,...
tt0087553,"['Sam Waterston', 'Haing S. Ngor', 'John Malko...",['\n\n Won\n 3\n ...,"$14,400,000"
tt6452574,"['Ranbir Kapoor', 'Paresh Rawal', 'Manisha Koi...",['\n 13 wins & 24 nominations.\n '],"$14,000,000"
tt2980516,"['Eddie Redmayne', 'Felicity Jones', 'Tom Prio...",['\n\n Won\n 1\n ...,"$15,000,000"


In [38]:
big_df = current_df.append(df)

In [39]:
big_df

,Actors,Awards,Budget
tt3040964,"['Neel Sethi', 'Bill Murray', 'Ben Kingsley', ...",['\n\n Won\n 1\n ...,"$175,000,000"
tt2310332,"['Ian McKellen', 'Martin Freeman', 'Richard Ar...",['\n\n Nominated for\n 1...,"$250,000,000"
tt1038988,"['Manuela Velasco', 'Ferran Terraza', 'Jorge-Y...",['\n 16 wins & 12 nominations.\n '],"EUR1,500,000"
tt0464141,"['Belén Rueda', 'Fernando Cayo', 'Roger Prínce...",['\n 32 wins & 43 nominations.\n '],"EUR3,400,000"
tt0451279,"['Gal Gadot', 'Chris Pine', 'Connie Nielsen', ...",['\n 25 wins & 71 nominations.\n '],"$149,000,000"
...,...,...,...
tt0101889,"[Jeff Bridges, Adam Bryant, Paul Lombardi, Dav...",[\n\n Won\n 1\n ...,"$24,000,000"
tt0097351,"[Kevin Costner, Amy Madigan, Gaby Hoffmann, Ra...",[\n\n Nominated for\n 3\...,NaN
tt0094721,"[Alec Baldwin, Geena Davis, Annie McEnroe, Mau...",[\n\n Won\n 1\n ...,"$15,000,000"
tt9484998,"[Andy Samberg, Cristin Milioti, J.K. Simmons, ...",[\n\n Nominated for\n 2\...,NaN


In [40]:
big_df.to_csv("../wth_actors_name/data/scraped_movie_data.csv")